In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
#from functools import reduce

pd.set_option('display.max_columns', 200)
pd.set_option('display.max_rows', 500)

In [2]:
pkls_path = pkls = r'D:\Tesis\ResEleCon-MX\pickles'
concentrador = pd.read_pickle(pkls_path + '/total_energy.pkl')

In [16]:
col = ['folioviv','foliohog','CVE_MUN','upm','est_socio','sexo_jefe','edad_jefe','educa_jefe','tot_integ','ing_cor',
       'num_cuarto','tenencia','gasto_tri_ele','gasto_tri_gas','gasto_tri_lpg','gasto_tri_total_gas','lpg_kwh',
       'g_natural_kwh','kwh','gas']
ener = concentrador[col].copy()

### Size original

In [38]:
len(ener)

31100

### Size without NAs in gas or electricity

In [40]:
ener.dropna(subset=['kwh','gas'], inplace=True)
len(ener)

18588

In [41]:
ener.isnull().sum(axis = 0)

folioviv                   0
foliohog                   0
CVE_MUN                    0
upm                        0
est_socio                  0
sexo_jefe                  0
edad_jefe                  0
educa_jefe                 0
tot_integ                  0
ing_cor                    0
num_cuarto                 0
tenencia                   0
gasto_tri_ele              0
gasto_tri_gas          15048
gasto_tri_lpg           3390
gasto_tri_total_gas        0
lpg_kwh                 3390
g_natural_kwh          15048
kwh                        0
gas                        0
dtype: int64

In [42]:
ener.to_pickle(pkls_path + '/energy_final.pkl')